##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to the Keras Tuner

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*. 

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [2]:
import tensorflow as tf
from tensorflow import keras

import IPython

Install and import the Keras Tuner.

In [3]:
#!pip install -q -U keras-tuner
import kerastuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist). 

Load the data.

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined `HyperModel` classes - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) and [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner. 

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. 

Before running the hyperparameter search, define a callback to clear the training outputs at the end of every training step.

In [8]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [9]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 128 and the optimal learning rate for the optimizer
is 0.001.



To finish this tutorial, retrain the model with the optimal hyperparameters from the search.

In [10]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - ETA: 0s - loss: 2.5947 - accuracy: 0.09 - ETA: 1s - loss: 1.1929 - accuracy: 0.59 - ETA: 1s - loss: 0.9877 - accuracy: 0.65 - ETA: 1s - loss: 0.8952 - accuracy: 0.69 - ETA: 1s - loss: 0.8237 - accuracy: 0.71 - ETA: 1s - loss: 0.7661 - accuracy: 0.73 - ETA: 1s - loss: 0.7246 - accuracy: 0.75 - ETA: 1s - loss: 0.6983 - accuracy: 0.75 - ETA: 1s - loss: 0.6797 - accuracy: 0.76 - ETA: 1s - loss: 0.6633 - accuracy: 0.77 - ETA: 1s - loss: 0.6467 - accuracy: 0.77 - ETA: 1s - loss: 0.6292 - accuracy: 0.78 - ETA: 1s - loss: 0.6157 - accuracy: 0.78 - ETA: 1s - loss: 0.6077 - accuracy: 0.78 - ETA: 1s - loss: 0.6025 - accuracy: 0.79 - ETA: 1s - loss: 0.5951 - accuracy: 0.79 - ETA: 1s - loss: 0.5907 - accuracy: 0.79 - ETA: 1s - loss: 0.5864 - accuracy: 0.79 - ETA: 1s - loss: 0.5819 - accuracy: 0.79 - ETA: 1s - loss: 0.5772 - accuracy: 0.80 - ETA: 1s - loss: 0.5747 - accuracy: 0.80 - ETA: 1s - loss: 0.5712 - accuracy: 0.80 - ETA: 1s - loss: 0.56

1875/1875 [==============================] - ETA: 0s - loss: 0.2375 - accuracy: 0.93 - ETA: 1s - loss: 0.2739 - accuracy: 0.89 - ETA: 1s - loss: 0.2940 - accuracy: 0.89 - ETA: 1s - loss: 0.2997 - accuracy: 0.88 - ETA: 1s - loss: 0.2928 - accuracy: 0.88 - ETA: 1s - loss: 0.2902 - accuracy: 0.88 - ETA: 1s - loss: 0.2890 - accuracy: 0.89 - ETA: 1s - loss: 0.2935 - accuracy: 0.88 - ETA: 1s - loss: 0.2935 - accuracy: 0.88 - ETA: 1s - loss: 0.2938 - accuracy: 0.88 - ETA: 1s - loss: 0.2953 - accuracy: 0.89 - ETA: 0s - loss: 0.2943 - accuracy: 0.89 - ETA: 0s - loss: 0.2938 - accuracy: 0.89 - ETA: 0s - loss: 0.2955 - accuracy: 0.89 - ETA: 0s - loss: 0.2978 - accuracy: 0.88 - ETA: 0s - loss: 0.2955 - accuracy: 0.89 - ETA: 0s - loss: 0.2954 - accuracy: 0.89 - ETA: 0s - loss: 0.2965 - accuracy: 0.89 - ETA: 0s - loss: 0.2973 - accuracy: 0.88 - ETA: 0s - loss: 0.2969 - accuracy: 0.88 - ETA: 0s - loss: 0.2959 - accuracy: 0.89 - ETA: 0s - loss: 0.2965 - accuracy: 0.89 - ETA: 0s - loss: 0.2965 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.1057 - accuracy: 0.96 - ETA: 2s - loss: 0.2438 - accuracy: 0.90 - ETA: 1s - loss: 0.2313 - accuracy: 0.91 - ETA: 1s - loss: 0.2340 - accuracy: 0.91 - ETA: 1s - loss: 0.2262 - accuracy: 0.91 - ETA: 1s - loss: 0.2305 - accuracy: 0.91 - ETA: 1s - loss: 0.2370 - accuracy: 0.91 - ETA: 1s - loss: 0.2391 - accuracy: 0.91 - ETA: 1s - loss: 0.2397 - accuracy: 0.91 - ETA: 1s - loss: 0.2388 - accuracy: 0.91 - ETA: 1s - loss: 0.2411 - accuracy: 0.90 - ETA: 1s - loss: 0.2422 - accuracy: 0.90 - ETA: 1s - loss: 0.2442 - accuracy: 0.90 - ETA: 1s - loss: 0.2422 - accuracy: 0.90 - ETA: 1s - loss: 0.2426 - accuracy: 0.90 - ETA: 1s - loss: 0.2424 - accuracy: 0.91 - ETA: 1s - loss: 0.2417 - accuracy: 0.91 - ETA: 1s - loss: 0.2436 - accuracy: 0.90 - ETA: 0s - loss: 0.2442 - accuracy: 0.90 - ETA: 0s - loss: 0.2454 - accuracy: 0.90 - ETA: 0s - loss: 0.2465 - accuracy: 0.90 - ETA: 0s - loss: 0.2469 - accuracy: 0.90 - ETA: 0s - loss: 0.2471 - accura

The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite = True` argument while instantiating the tuner.